In [1]:
#!pip install faiss

In [1]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv


/home/domino/.local/lib/python3.11/site-packages/pydantic/plugin/_loader.py:49: UserWarning: ImportError while loading the `logfire-plugin` Pydantic plugin, this plugin will not be installed.

ImportError("cannot import name 'OTEL_EXPORTER_OTLP_CLIENT_CERTIFICATE' from 'opentelemetry.sdk.environment_variables' (/home/domino/.local/lib/python3.11/site-packages/opentelemetry/sdk/environment_variables.py)")
  warnings.warn(


In [2]:
# texts = [
#     "The iPhone 15 Pro costs $999 and comes with 128GB of storage. It's available on our Unlimited Plus plan for $34.99 per month for 36 months.",
#     "The Samsung Galaxy S24 Ultra is priced at $1,299 and features 512GB of storage. You can get it for $36.11 per month with our Unlimited Plus plan.",
#     "The Google Pixel 8 Pro is available for $899 with 256GB of memory. Monthly payments are $24.99 for 36 months on the Unlimited Welcome plan.",
#     "Your current bill is $157.32 which includes $34.99 for your iPhone 15 Pro payment and $80 for two lines on the Unlimited Plus plan.",
#     "The Motorola Razr+ foldable phone costs $999 with 256GB of storage. It can be purchased for $27.77 per month for 36 months.",
#     "Your Samsung Galaxy S24 Ultra has 512GB of storage which is sufficient for approximately 100,000 photos or 70 hours of 4K video recording.",
#     "The Google Pixel 8 includes 128GB of memory and costs $699 or $19.44 per month for 36 months.",
#     "Your next bill will be $162.45 due on March 25th, which includes the monthly payment for your Pixel 8 Pro and two lines on Unlimited Welcome.",
#     "The iPhone 15 has 128GB of storage and costs $799 or $22.19 per month for 36 months on our payment plan.",
#     "The Samsung Galaxy Z Fold 5 with 512GB memory is our premium foldable device at $1,799 or $49.97 per month for 36 months.",
#     "Your iPhone 15 Pro in Titanium Blue with 256GB storage costs $1,099 or $30.53 per month for 36 months.",
#     "The Google Pixel 8 Pro has an excellent camera system and 256GB of storage for all your photos and videos.",
#     "Your account currently has 3 lines: two phone lines on Unlimited Plus for $80 and one tablet line for $10 per month.",
#     "The Samsung Galaxy S24 is our standard model with 256GB storage priced at $799 or $22.19 monthly for 36 months.",
#     "Your iPhone 15 Pro Max has 1TB of storage and costs $1,599 or $44.42 per month on the device payment plan."
# ]

In [3]:
# Load CSV data
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# Chunking based on payment frequency
def chunk_data(df, group_column='Payment Frequency'):
    grouped = df.groupby(group_column)
    chunks = []
    metadata = []
    for name, group in grouped:
        chunk_text = '\n'.join(group.apply(lambda x: str(x.to_dict()), axis=1))
        chunks.append(chunk_text)
        metadata.append({'Payment Frequency': name})
    return chunks, metadata

def chunk_updated(df):
    chunks = df.apply(lambda x: str(x.to_dict()),axis=1).to_list()
    return chunks


In [4]:
import pandas as pd
file_path = 'Data.csv'  # Update with your file path
df = load_data(file_path)
chunks = chunk_updated(df)

In [22]:
# Initialize the embedding model
print("Loading embedding model...")
s_model = SentenceTransformer('BAAI/bge-small-en-v1.5')

Loading embedding model...


In [6]:
# Create embeddings for the text data
print("Creating embeddings...")
embeddings = s_model.encode(chunks, normalize_embeddings=True)
embeddings = np.array(embeddings).astype('float32')

Creating embeddings...


In [7]:
# Create a FAISS index
print("Creating FAISS index...")
dimension = embeddings.shape[1]  # Get the dimension of embeddings
index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)

Creating FAISS index...


In [8]:
dimension

384

In [9]:
index

<faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7b1a9f532100> >

In [10]:
# Add the vectors to the index
index.add(embeddings)
print(f"Added {index.ntotal} vectors of dimension {dimension} to the index")

Added 7 vectors of dimension 384 to the index


In [23]:
def search_db(query_text,index, top_k=1):
    # Convert query to embedding
    query_embedding = s_model.encode([query_text], normalize_embeddings=True)
    query_embedding = np.array(query_embedding).astype('float32')

    # Search the index
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "text": chunks[idx],
            "distance": float(distances[0][i]),
            "index": int(idx)
        })

    return results

In [12]:
# sample_queries = [
#     "How much does the iPhone 15 Pro cost?",
#     "What is the storage capacity of the Samsung Galaxy S24 Ultra?",
#     "How much is my monthly bill?",
#     "What is the memory of the Google Pixel 8 Pro?",
#     "How much is the monthly payment for the Samsung Galaxy S24?"
# ]

In [24]:
sample_queries = [
#     "Extract monthly price under 40 dollar",
    "explain golden plan"
    
]

In [25]:
for query in sample_queries:
    print(f"\nQuery: '{query}'")
    results = search_db(query,index)
#     print("Top results:",results['text'])
    print()
    s = ''
    for i, result in enumerate(results):
#         print(f"{i+1}. [{result['distance']:.4f}] {result['text']}")
        s = s + result['text']
    print(s)


Query: 'explain golden plan'

{'Payment Frequency': '1 Month', 'FCC Heading': 'Broadband Facts', 'Provider Name': 'Straight Talk Wireless', 'Plan Name': 'Gold Unlimited', 'Plan Type': 'Mobile Broadband Consumer Disclosure', 'Monthly Price': '$55.00', 'Monthly Introductory Part': 'This Monthly Price is not an introductory rate.This Monthly Price does not require a contract.', 'Activation Fee': '$0.00', 'Provider Monthly Fees': '$0.00', 'One-Time Fees at the Time of Purchase': ' ', 'Federal Universal Service Fund': '$0.30', 'Regulatory Cost Recovery': '$0.11', 'Early Termination Fee': '$0.00', 'Government Taxes': 'Varies by location', 'Discounts & Bundles': 'Click here\xa0for available discounts and pricing options for broadband service bundled with other services like video, phones, and wireless services, and use of your own equipment like modem or routers.', '5G-Typical Download Speed': '35-143 Mbps', '5G-Typical Upload Speed': '5-31 Mbps', '5G-Typical Latency': '42-64 ms', '4G LTE-Ty

In [26]:
def generate_answer(model, tokenizer, context, question):
    """Generates an answer using the Gemma 3 1B model."""
    prompt = f"""
    Context:
    {context}
    
    Question:
    {question}
    
    Answer:
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    output = model.generate(**inputs, max_length=900,temperature=0.005,do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)
def load_gemma_model():
    """Loads the Gemma 3 1B model and tokenizer."""
    model_name = "google/gemma-3-1b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
    return model, tokenizer
def user_input(user_question,model, tokenizer):
    """Processes user input and retrieves an answer."""

    import time
    st = time.time()
    results = search_db(user_question,index)
    s_result = ''
    for i, result in enumerate(results):
#         print(f"{i+1}. [{result['distance']:.4f}] {result['text']}")
        s_result = s_result + result['text']

    print("similarity result",s_result)
    et = time.time()
    print("total time taken for fetcing from vector db",round(et-st,2))
    print()
    print()
    
    st = time.time()
    response = generate_answer(model, tokenizer, s_result, user_question)
    et = time.time()
    print("total time taken for llm",round(et-st,2))
    
    print("Gemma 3 1B Response:", response)
    
model, tokenizer = load_gemma_model()

In [28]:
user_question = input("Ask a question (or type 'exit' to quit): ")
user_input(user_question,model, tokenizer)

Ask a question (or type 'exit' to quit): explain golden plan
similarity result {'Payment Frequency': '1 Month', 'FCC Heading': 'Broadband Facts', 'Provider Name': 'Straight Talk Wireless', 'Plan Name': 'Gold Unlimited', 'Plan Type': 'Mobile Broadband Consumer Disclosure', 'Monthly Price': '$55.00', 'Monthly Introductory Part': 'This Monthly Price is not an introductory rate.This Monthly Price does not require a contract.', 'Activation Fee': '$0.00', 'Provider Monthly Fees': '$0.00', 'One-Time Fees at the Time of Purchase': ' ', 'Federal Universal Service Fund': '$0.30', 'Regulatory Cost Recovery': '$0.11', 'Early Termination Fee': '$0.00', 'Government Taxes': 'Varies by location', 'Discounts & Bundles': 'Click here\xa0for available discounts and pricing options for broadband service bundled with other services like video, phones, and wireless services, and use of your own equipment like modem or routers.', '5G-Typical Download Speed': '35-143 Mbps', '5G-Typical Upload Speed': '5-31 Mbp

KeyboardInterrupt: Interrupted by user

In [ ]:
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
#     docs = new_db.similarity_search(user_question)